# LeNet

In [1]:
%matplotlib inline

from collections import defaultdict

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import torchvision
import torchvision.transforms as transforms
from torchvision.transforms import RandomCrop, RandomRotation
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns


plt.rcParams['figure.figsize'] = (9, 6)

sns.set_style('darkgrid')

In [2]:
DATA_DIR = "../data"

In [3]:
def num_flat_features(self, x):
    """return the number of flat features from a pytorch variable"""
    return int(np.prod(x.size()[1:]))


## Defining the model

In [4]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()  # run initializer on the parent class
        
        # Convolutional Layers
        # 1 image, 6 output channels, 5x5 convolution
        self.conv1 = nn.Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
        self.conv2 = nn.Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))

        # Fully Connected Layers
        self.fc1 = nn.Linear(256, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        """
        forward must be overwritten in torch model class
        """
        # Convolutional Layers
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))  # add pooling layer
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        
        x = x.view(-1, 256)  # flatten for fully connected layers

        # fully connected layers
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x
    
    def __str__(self):
        return "LeNet"


In [5]:
net = LeNet()
print(net)

LeNet


Define loss and optimization algorithms

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters())

## Data handling

In [7]:
def load_data(training=True):
    transform_ = transforms.Compose(
        [RandomRotation(45),
         RandomCrop(28),
         transforms.ToTensor()]
    )
    data = torchvision.datasets.MNIST(
        root='./data/',
        train=training,
        download=True,
        transform=transform_,
    )
    loader = torch.utils.data.DataLoader(
        data,
        batch_size=16,
        shuffle=True,
        num_workers=2,
    )
   
    return loader

Load the data.
If the data files do not exist, download them.

In [8]:
dataloader = load_data(training=True)

In [9]:
def train(net, dataloader, criterion, optimizer, verbose=True):
    scores = []
    running_loss = 0.0
    correct = 0
    total = 0

    verbose = True

    for i, (inputs, labels) in enumerate(dataloader, 0):
        # wrap features as torch Variables
        inputs, labels = Variable(inputs), Variable(labels)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs) # forward pass
        loss = criterion(outputs, labels)  # optimization
        loss.backward()  # compute back propagation
        optimizer.step()  # update model parameters

        running_loss += loss.data[0]

        if i % 100 == 99:  # print every 2000 mini-batches
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += predicted.eq(labels.data).cpu().sum()
            accuracy = 100. * correct / total

            scores.append((i+1, running_loss/100, accuracy)) 

            # print results
            if verbose and i % 500 == 499:
                print('Batch: %5d - Loss: %.3f' % (i+1, running_loss/100))
                print("Accuracy: {:.2f}%".format(accuracy))

            running_loss = 0.0    

    print('Finished Training')
    
    return scores

In [10]:
def unpack_data(data):
    iterations = [i[0] for i in scores]
    loss_scores = [i[1] for i in scores]
    acc_scores = [i[2] for i in scores]
    
    return iterations, loss_scores, acc_scores


In [11]:
results = defaultdict(list)

for i in range(5):  # number of models
    print("Model {}...".format(i+1))
    net = LeNet()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(net.parameters())
    
    scores = train(
        net,  # the model
        dataloader,  # the data provider
        criterion,  # the loss function
        optimizer,  # the optimization algorithm
        verbose=True,  # print results
    )
    step, loss, acc = unpack_data(scores) 
    
    net.zero_grad()
    optimizer.zero_grad()
    
    results['step'] += step
    results['loss_scores'] += loss
    results['acc_scores'] += acc
    results['model'] += [i] * len(step)
        
    del net, criterion, optimizer
    
results = dict(results)
    

Model 1...
Batch:   500 - Loss: 0.496
Accuracy: 83.75%
Batch:  1000 - Loss: 0.308
Accuracy: 83.75%
Batch:  1500 - Loss: 0.239
Accuracy: 85.00%
Batch:  2000 - Loss: 0.238
Accuracy: 87.19%
Batch:  2500 - Loss: 0.219
Accuracy: 88.50%
Batch:  3000 - Loss: 0.191
Accuracy: 88.54%
Batch:  3500 - Loss: 0.151
Accuracy: 89.29%
Finished Training


In [13]:
def plot_loss(i, loss):
    plt.plot(i, loss);
    plt.title('Loss')
    plt.xlabel('Step')
    plt.y_label('Model Loss')


In [14]:
def plot_accuracy(i, acc):
    plt.plot(i, acc);
    plt.title('Accuracy')
    plt.xlabel('Step')
    plt.y_label('Accuracy')


# Results

In [15]:
df = pd.DataFrame.from_dict(dict(results))

In [17]:
df.head(50)

,acc_scores,loss_scores,model,step
0,75.000000,1.820343,0,100
1,75.000000,0.896007,0,200
2,79.166667,0.708093,0,300
3,81.250000,0.564211,0,400
4,83.750000,0.496061,0,500
5,83.333333,0.383070,0,600
6,83.035714,0.382555,0,700
7,84.375000,0.327387,0,800
8,84.722222,0.333873,0,900
9,83.750000,0.307877,0,1000
